# 1. Import the Preprocessed Data
Import the pickle file with the preprocessed images.

In [1]:
import pickle
import requests
import io
import numpy as np
import pandas as pd

pickle_file = 'https://static.bc-edx.com/ai/ail-v-1-0/m19/lesson_3/datasets/pickles/preprocessed_faces_data.pkl'
response = requests.get(pickle_file)
data = pd.read_pickle(io.BytesIO(response.content))
data.keys()

dict_keys(['X_train', 'X_test', 'y_train_userid', 'y_train_pose', 'y_train_expression', 'y_train_eyes', 'y_test_userid', 'y_test_pose', 'y_test_expression', 'y_test_eyes'])

In [2]:
X_train = data['X_train']
X_test = data['X_test']

y_train_userid = data['y_train_userid']
y_train_pose = data['y_train_pose']
y_train_expression = data['y_train_expression']
y_train_eyes = data['y_train_eyes']

y_test_userid = data['y_test_userid']
y_test_pose = data['y_test_pose']
y_test_expression = data['y_test_expression']
y_test_eyes = data['y_test_eyes']

# 2. Build the Model

In [3]:
from tensorflow.keras import layers, models, Model
# First we build the input layer
input_layer = layers.Input(shape=(60, 64, 1), name='input_layer')

# Shared layers (common across all tasks)
# The second layer should be a Conv2D layer built off the input_layer
conv1 = layers.Conv2D(32, (3, 3), activation='relu')(input_layer)

# The third layer should be a MaxPooling2D layer built off the second layer
maxpool1 = layers.MaxPooling2D((2, 2))(conv1)

# The fourth layer should be a Conv2D layer built off the third layer
conv2 = layers.Conv2D(64, (3, 3), activation='relu')(maxpool1)

# The fifth layer should be a MaxPooling2D layer built off the fourth layer
maxpool2 = layers.MaxPooling2D((2, 2))(conv2)

# The sixth layer should be a Conv2D layer built off the fifth layer
conv3 = layers.Conv2D(64, (3, 3), activation='relu')(maxpool2)

# The seventh layer should be a Flatten layer built off the sixth layer
flatten = layers.Flatten()(conv3)

# Lastly, build one dense layer before branching to the different y branches
dense_shared = layers.Dense(64, activation='relu')(flatten)


In [4]:
# Build the branches for each of the y variables
# Include a dense hidden layer in each along with the output layer.
# Remember to include the correct number of nodes for the output!

# userid
userid_dense = layers.Dense(64, activation='relu')(dense_shared)
userid_output = layers.Dense(len(y_train_userid.columns),
                             activation='sigmoid',
                             name='userid_output')(userid_dense)

# pose
pose_dense = layers.Dense(64, activation='relu')(dense_shared)
pose_output = layers.Dense(len(y_train_pose.columns),
                           activation='softmax',
                             name='pose_output')(pose_dense)

# expression
expression_dense = layers.Dense(64, activation='relu')(dense_shared)
expression_output = layers.Dense(len(y_train_expression.columns),
                                 activation='softmax',
                             name='expression_output')(expression_dense)

# eyes
eyes_dense = layers.Dense(64, activation='relu')(dense_shared)
eyes_output = layers.Dense(len(y_train_eyes.columns),
                           activation='sigmoid',
                             name='eyes_output')(eyes_dense)

In [5]:
# Build the model
model = Model(inputs=input_layer, outputs=[
    userid_output,
    pose_output,
    expression_output,
    eyes_output
])

# Compile the model
model.compile(optimizer='adam',
              loss={'userid_output': 'categorical_crossentropy',
                    'pose_output': 'categorical_crossentropy',
                    'expression_output': 'categorical_crossentropy',
                    'eyes_output': 'binary_crossentropy'},
              metrics={'userid_output': 'accuracy',
                       'pose_output': 'accuracy',
                       'expression_output': 'accuracy',
                       'eyes_output': 'accuracy'})

In [6]:
# Train the model with the training data
model.fit(
    X_train,
    {
        'userid_output': y_train_userid,
        'pose_output': y_train_pose,
        'expression_output': y_train_expression,
        'eyes_output': y_train_eyes
    },
    epochs=10,  # You can adjust the number of epochs based on your needs
    batch_size=32,  # You can adjust the batch size based on your available memory
    validation_split=0.2  # You can specify the validation split if you have a separate validation set
)

Epoch 1/10
59/59 ━━━━━━━━━━━━━━━━━━━━ 15s 99ms/step - expression_output_accuracy: 0.2929 - eyes_output_accuracy: 0.5116 - loss: 6.4403 - pose_output_accuracy: 0.2776 - userid_output_accuracy: 0.0630 - val_expression_output_accuracy: 0.2692 - val_eyes_output_accuracy: 0.7500 - val_loss: 5.9636 - val_pose_output_accuracy: 0.4167 - val_userid_output_accuracy: 0.1603
Epoch 2/10
59/59 ━━━━━━━━━━━━━━━━━━━━ 9s 83ms/step - expression_output_accuracy: 0.2849 - eyes_output_accuracy: 0.7141 - loss: 5.4154 - pose_output_accuracy: 0.4189 - userid_output_accuracy: 0.3246 - val_expression_output_accuracy: 0.2436 - val_eyes_output_accuracy: 0.7735 - val_loss: 4.9836 - val_pose_output_accuracy: 0.4872 - val_userid_output_accuracy: 0.3932
Epoch 3/10
59/59 ━━━━━━━━━━━━━━━━━━━━ 5s 90ms/step - expression_output_accuracy: 0.3198 - eyes_output_accuracy: 0.7598 - loss: 4.3680 - pose_output_accuracy: 0.5163 - userid_output_accuracy: 0.5410 - val_expression_output_accuracy: 0.2009 - val_eyes_output_accuracy: 0.

In [7]:
# Evaluate the model using the testing data
results = model.evaluate(np.array(X_test), {
        'userid_output': y_test_userid,
        'pose_output': y_test_pose,
        'expression_output': y_test_expression,
        'eyes_output': y_test_eyes
    })

5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step - expression_output_accuracy: 0.1780 - eyes_output_accuracy: 0.8699 - loss: 4.1096 - pose_output_accuracy: 0.7608 - userid_output_accuracy: 0.8880
